<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Getting-valid-URL's" data-toc-modified-id="Getting-valid-URL's-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting valid URL's</a></span><ul class="toc-item"><li><span><a href="#Check-for-valid-url's" data-toc-modified-id="Check-for-valid-url's-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Check for valid url's</a></span></li><li><span><a href="#Clean-categories" data-toc-modified-id="Clean-categories-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Clean categories</a></span></li><li><span><a href="#Add-reliable-URL's" data-toc-modified-id="Add-reliable-URL's-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Add reliable URL's</a></span></li><li><span><a href="#Save-the-new-url-list" data-toc-modified-id="Save-the-new-url-list-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Save the new url list</a></span></li></ul></li><li><span><a href="#Scrapy/BeautifulSoup" data-toc-modified-id="Scrapy/BeautifulSoup-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scrapy/BeautifulSoup</a></span></li><li><span><a href="#Web-Scraping-(nytimes)" data-toc-modified-id="Web-Scraping-(nytimes)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Web Scraping (nytimes)</a></span><ul class="toc-item"><li><span><a href="#Archive-search" data-toc-modified-id="Archive-search-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Archive search</a></span></li><li><span><a href="#Article-search" data-toc-modified-id="Article-search-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Article search</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Example</a></span></li></ul></li><li><span><a href="#News-API" data-toc-modified-id="News-API-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>News API</a></span></li><li><span><a href="#webhose.io" data-toc-modified-id="webhose.io-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>webhose.io</a></span><ul class="toc-item"><li><span><a href="#API-Plyground" data-toc-modified-id="API-Plyground-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>API Plyground</a></span></li><li><span><a href="#Python-API" data-toc-modified-id="Python-API-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Python API</a></span></li></ul></li><li><span><a href="#Extracting-extra-data-from-a-dataset-found-at-webhose.io" data-toc-modified-id="Extracting-extra-data-from-a-dataset-found-at-webhose.io-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Extracting extra data from a dataset found at webhose.io</a></span><ul class="toc-item"><li><span><a href="#URL's" data-toc-modified-id="URL's-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>URL's</a></span></li><li><span><a href="#Select-only-sites-with-.com-suffix" data-toc-modified-id="Select-only-sites-with-.com-suffix-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Select only sites with <code>.com</code> suffix</a></span></li><li><span><a href="#Remove-none-US-sites" data-toc-modified-id="Remove-none-US-sites-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Remove none-US sites</a></span></li><li><span><a href="#Collect-news-article" data-toc-modified-id="Collect-news-article-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Collect news article</a></span></li><li><span><a href="#Save-content" data-toc-modified-id="Save-content-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Save content</a></span></li></ul></li></ul></div>

## Libraries

In [12]:
import os
import re
import sys
import json
import time
import requests
import ujson
import random
import socket
import tldextract
import geoip2.database
import socket
import webhose
import datetime
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from pathlib import Path
from urllib.request import urlopen
from newsapi import NewsApiClient

Load the file with all the needed api tokens.

In [3]:
config_file = os.path.join(str(Path.home()), '.auth/config.json')
with open(config_file) as f:
    tokens = json.load(f)

---

## Getting valid URL's

[OpenSources](http://www.opensources.co/) is a curated resource for assessing online information sources, available for public use. Websites in this resource range from credible news sources to misleading and outright fake websites. 

In the given csv file we have over 800 url's.

In [2]:
source_file = 'data/sources.csv'

df = pd.read_csv(source_file)
df.head()

,Unnamed: 0,type,2nd type,3rd type,Source Notes (things to know?),Unnamed: 5
0,100percentfedup.com,bias,NaN,NaN,NaN,NaN
1,365usanews.com,bias,conspiracy,NaN,NaN,NaN
2,4threvolutionarywar.wordpress.com,bias,conspiracy,NaN,NaN,NaN
3,aheadoftheherd.com,bias,NaN,NaN,"false quotes regarding banking, heavily promot...",NaN
4,americablog.com,bias,clickbait,NaN,domain for sale,NaN


In [3]:
df.shape

(832, 6)

We wil rename column `Unnamed: 0` to `url` and `type` to `category`, and drop all the other columns. We could look more closely at `2nd type`, but for now we will focus on looking at the url's given.

In [4]:
df.drop(['2nd type', '3rd type', 'Source Notes (things to know?)', 'Unnamed: 5'], axis=1, inplace=True)

In [5]:
df.columns = ['url', 'category']
df.head()

,url,category
0,100percentfedup.com,bias
1,365usanews.com,bias
2,4threvolutionarywar.wordpress.com,bias
3,aheadoftheherd.com,bias
4,americablog.com,bias


### Check for valid url's

Let us check if we can open the url's in the list provided and if not remove it from the dataset.

In [6]:
with tqdm() as progress:
    for url in df['url']:
        try:
            code = urlopen('http://' + url).getcode()
        except IOError:
            df = df[df.url != url]
            code = None
            
        progress.update()

832it [22:33,  1.09s/it]


In [7]:
df.shape

(472, 2)

From the orginal 832 url's we have reduced it down to 472.

In [8]:
df['category'].value_counts()

fake          134
conspiracy     75
satire         72
bias           67
political      40
unreliable     30
junksci        17
hate           13
clickbait      12
rumor           4
reliable        3
rumor           1
fake news       1
fake            1
state           1
Conspiracy      1
Name: category, dtype: int64

We have an overwhelmingly large number of url's which are categorized as fake. As well as consipacy, bias and unreliable. Compare to only 3 reliable url's. 

### Clean categories

From the looks we have some misspelling in the naming of the categories. Let's clean it.

In [9]:
df['category'].unique()

array(['bias', 'clickbait', 'conspiracy', 'Conspiracy', 'fake', 'fake ',
       'fake news', 'hate', 'junksci', 'satire', 'political', 'reliable',
       'rumor', 'rumor ', 'state', 'unreliable'], dtype=object)

In [10]:
df = df[(df['category'] != 'rumor') & (df['category'] != 'rumor ') & 
        (df['category'] != 'hate') & (df['category'] != 'state') &
        (df['category'] != 'Conspiracy') & (df['category'] != 'fake news') & 
        (df['category'] != 'fake ')]

print(df['category'].unique())
print(df['category'].value_counts())

['bias' 'clickbait' 'conspiracy' 'fake' 'junksci' 'satire' 'political'
 'reliable' 'unreliable']
fake          134
conspiracy     75
satire         72
bias           67
political      40
unreliable     30
junksci        17
clickbait      12
reliable        3
Name: category, dtype: int64


### Add reliable URL's

Because of the very low number of reliable url's we will add a few of our own.

In [11]:
domains_reliable = ['reuters.com', 'yahoo.com', 'sfgate.com', 'chron.com', 'news.yahoo.com',
                    'uk.finance.yahoo.com', 'huffingtonpost.com', 'abcnews.go.com', 'msn.com', 
                    'businessinsider.com', 'uk.reuters.com', 'theguardian.com', 'buzzfeed.com', 
                    'ca.reuters.com', 'wsj.com', 'af.reuters.com', 'nhl.com', 'legacy.com', 
                    'latimes.com', 'washingtonpost.com', 'sports.yahoo.com', 'forbes.com', 
                    'nytimes.com', 'nydailynews.com', 'bloomberg.com', 'in.reuters.com', 
                    'usatoday.com', 'ndtv.com', 'music.yahoo.com', 'cnbc.com', 'ca.news.yahoo.com', 
                    'cbsnews.com', 'marketwatch.com', 'm.mlb.com', 'nypost.com', 'npr.org', 
                    'bleacherreport.com', 'finance.yahoo.com', 'feeds.reuters.com', 'cbssports.com', 
                    'edition.cnn.com', 'people.com', 'cnet.com', 'nz.sports.yahoo.com', 'news.abs-cbn.com',
                    'nba.com', 'online.wsj.com', 'investing.com', 'cnn.com', 'eonline.com', 
                    'newsinfo.inquirer.net', 'nbcnews.com', 'politico.com', 'profit.ndtv.com', 
                    'uk.news.yahoo.com', 'video.foxnews.com', 'au.finance.yahoo.com', 'nz.news.yahoo.com', 
                    'engadget.com', 'vice.com', 'wikihow.com', 'money.cnn.com', 'nfl.com', 
                    'washpost.bloomberg.com', 'realclearpolitics.com', 'motherboard.vice.com', 
                    'au.news.yahoo.com', 'theatlantic.com']

print(len(domains_reliable))

68


In [12]:
reliable_urls = [item for item in domains_reliable if item not in df.url]
print(len(reliable_urls))

68


In [13]:
reliable_df = pd.DataFrame(data=reliable_urls, columns=['url'])
reliable_df['category'] = 'reliable'
reliable_df.tail()

,url,category
63,washpost.bloomberg.com,reliable
64,realclearpolitics.com,reliable
65,motherboard.vice.com,reliable
66,au.news.yahoo.com,reliable
67,theatlantic.com,reliable


In [14]:
df = df.append(reliable_df, ignore_index=True)
print(df['category'].value_counts())

fake          134
conspiracy     75
satire         72
reliable       71
bias           67
political      40
unreliable     30
junksci        17
clickbait      12
Name: category, dtype: int64


In [15]:
df.shape

(518, 2)

In [16]:
print(len(df['category'].unique()))

9


That looks somewhat better now. Still off balance but good enough for us to start gathering some content.

### Save the new url list

In [17]:
df.to_csv('data/url_sources.csv', index=False)

---

## Scrapy/BeautifulSoup

The attempt to get the news content from a website can be a daunting task. Especially when we are looking at an article from the nytimes. Instead of having the article contained in one html tag, here we find each paragraph nested in it's own `<p class>`. Which makes it not necessarily difficult but just bloody hard.

Now if we multiply the task of webscarping by the amount of url's we got in our list we will be spending some time on it. More then we probably like.

Let us explore other options.

---

## Web Scraping (nytimes)

The nytimes offers an api to search their archive, articles and more to get the web url's to the articles as well as headlines, document type etc.

### Archive search

In [ ]:
url = 'https://api.nytimes.com/svc/archive/v1/%s/%s.json?api-key=%s'

begin_year = 2017
end_year = 2018

with open('nytimes_data/archivesearch.json', 'w') as f:
    with tqdm() as progress:
        for year in range(begin_year, end_year + 1):
            for month in range(1, 13):
                try:
                    response = requests.get(url % (year, month, NYTIMES_TOKEN))
                    docs = response.json()['response']['docs']
                except Exception:
                    print('Failed to get {}/{}'.format(year, month))
                
                # There are two rate limits per API: 4,000 requests per day and 10 requests per minute. 
                # You should sleep 6 seconds between calls to avoid hitting the per minute rate limit.
                time.sleep(6) 
                
                progress.update()
                
    f.write(ujson.dumps(docs) + '\n')

In [50]:
data = pd.read_json('nytimes_data/archivesearch.json')
data.head()

,_id,blog,byline,document_type,headline,keywords,multimedia,news_desk,print_page,pub_date,score,section_name,slideshow_credits,snippet,source,type_of_material,uri,web_url,word_count
0,5c04323300a1bc2872e96b1a,[],{'original': 'By BARBARA MARCOLINI and NEETI U...,multimedia,{'main': 'The People ‘Are Hungry’: Scenes from...,"[{'name': 'subject', 'value': 'Demonstrations,...","[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",World / Europe,NaN,2018-12-02T19:27:44+0000,1,Europe,NaN,Ambulance workers and students joined anti-gov...,The New York Times,Video,nyt://video/f3ee5687-4bd3-5224-a9b1-41dd912520fb,https://www.nytimes.com/video/world/europe/100...,0
1,5c0269b300a1bc2872e96926,[],"{'original': 'By FRANK DeCARO', 'person': [{'f...",article,"{'main': 'The Marvelous Mr. Mackie', 'kicker':...","[{'name': 'persons', 'value': 'Mackie, Bob', '...","[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",Styles,1.0,2018-12-01T11:00:01+0000,1,NaN,NaN,"Thanks to a new musical, Cher’s favorite desig...",The New York Times,News,nyt://article/902519e6-21c3-5441-857d-522efe10...,https://www.nytimes.com/2018/12/01/style/cher-...,2820
2,5c02911b00a1bc2872e96945,[],"{'original': 'By SHASTA DARLINGTON', 'person':...",article,{'main': 'Brazil’s New Leader Wants to Ease Gu...,"[{'name': 'subject', 'value': 'Gun Control', '...","[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",Foreign,14.0,2018-12-01T13:48:09+0000,1,Americas,NaN,President-elect Jair Bolsonaro wants to make i...,The New York Times,News,nyt://article/e619c196-fd4d-5e5e-b463-3a0df52e...,https://www.nytimes.com/2018/12/01/world/ameri...,1221
3,5c0226ea00a1bc2872e968e3,[],"{'original': 'By THE EDITORIAL BOARD', 'person...",article,"{'main': 'George H.W. Bush, Public Servant', '...","[{'name': 'persons', 'value': 'Bush, George', ...","[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",Editorial,26.0,2018-12-01T06:15:02+0000,1,NaN,NaN,"He made his share of mistakes, but he worked f...",The New York Times,Editorial,nyt://article/c1f5796b-f0cd-541f-86b0-37b00475...,https://www.nytimes.com/2018/12/01/opinion/geo...,1187
4,5c022a6800a1bc2872e968ea,[],{'original': 'By SARAH MERVOSH and JACEY FORTI...,article,{'main': 'Politicians and Family React to Geor...,"[{'name': 'persons', 'value': 'Bush, George', ...","[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",Express,27.0,2018-12-01T06:29:59+0000,1,NaN,NaN,"Politicians, including President Trump, joined...",The New York Times,News,nyt://article/a830058e-7478-59c8-a41c-c924ad2e...,https://www.nytimes.com/2018/12/01/us/george-b...,1066


### Article search

In [2]:
def get_date_range(start_date, end_date):
    '''
    Return date between start and end date
    '''
    if start_date <= end_date:
        for i in range((end_date - start_date).days + 1):
            yield start_date + datetime.timedelta(i)
    else:
        for i in range((start_date - end_date).days + 1):
            yield start_date - datetime.timedelta(i)

In [7]:
def get_json_file(date, page, json_file_path):
    '''
    Define file name
    '''
    json_file_name = ".".join([str(date), str(page), 'json'])
    json_file_name = "/".join([json_file_path, json_file_name])
    
    return json_file_name

In [10]:
def get_articles(date, query, json_file_path):
    '''
    Get all articles from the NYTimes Article Search API
    
    The Article Search API returns a max of 10 results at a time. The meta 
    node in the response contains the total number of matches ("hits") and 
    the current offset. Use the page query parameter to paginate thru results 
    (page=0 for results 1-10, page=1 for 11-20, ...). You can paginate thru 
    up to 100 pages (1,000 results). If you get too many results try filtering 
    by date range.
    '''
    
    for page in range(0, 100):
        # recommented wait time between requests
        time.sleep(6) 
        
        request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=" + query + "&begin_date=" + \
                            str(date) + "&end_date=" + str(date) + "&page=" + str(page) + "&api-key=" + NYTIMES_TOKEN
        response = requests.get(request_string)
        if response.json()['status'] == 'OK':
            docs = response.json()['response']['docs']
            if len(docs) >= 1:
                json_file_name = get_json_file(date, page, json_file_path)
                with open(json_file_name, 'w') as f:
                    f.write(ujson.dumps(docs) + '\n')
                    f.close()
            # if no more articles, go to next date
            else:
                return

Let's search December 2018 and January 2019 for an arbitrary topic.

In [11]:
query = 'Goverment+Shutdown'
json_file_path = 'json_data'

# define start date
start_year = 2018
start_month = 12
start_day = 1

# define end date
end_year = 2019
end_month = 1
end_day = 31

# define date
start_date = datetime.date(year = start_year, month = start_month, day = start_day)
end_date = datetime.date(year = end_year, month = end_month, day = end_day)

with tqdm() as progress:
    for date in get_date_range(start_date, end_date):
        date = date.strftime("%Y%m%d")
        get_articles(date, query, json_file_path)

        progress.update()

62it [07:13,  6.89s/it]


In [17]:
data = pd.read_json('json_data/20181222.0.json')
data.head()

,_id,blog,byline,document_type,headline,keywords,lead_paragraph,multimedia,news_desk,print_page,pub_date,score,section_name,snippet,source,subsectoinName,type_of_material,uri,web_url,word_count
0,5c1e759d3a125f5075c01911,{},{'original': 'By SHERYL GAY STOLBERG and KATIE...,article,{'main': 'Government Shutdown to Continue for ...,"[{'name': 'persons', 'value': 'Trump, Donald J...",Senate Democrats said they would never accede ...,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",Washington,1,2018-12-22T17:33:58+0000,0,U.S.,Senate Democrats said they would never accede ...,The New York Times,Politics,News,nyt://article/f84eebb0-8ca8-5a24-b543-344ea5e2...,https://www.nytimes.com/2018/12/22/us/politics...,1706


In [18]:
data.columns

Index(['_id', 'blog', 'byline', 'document_type', 'headline', 'keywords',
       'lead_paragraph', 'multimedia', 'news_desk', 'print_page', 'pub_date',
       'score', 'section_name', 'snippet', 'source', 'subsectoinName',
       'type_of_material', 'uri', 'web_url', 'word_count'],
      dtype='object')

In [20]:
with open('json_data/20181222.0.json') as f:
    data = json.load(f)
    print(data[0]['headline']['main'] + '\n')
    print(data[0]['lead_paragraph'] + '\n')
    print(data[0]['web_url'] + '\n')

Government Shutdown to Continue for Days as Senate Adjourns Until Thursday

Senate Democrats said they would never accede to President Trump’s demand for $5 billion for a border wall, and the White House offered no early indication that the president would accept less.

https://www.nytimes.com/2018/12/22/us/politics/shutdown-trump-senate.html



NYTimes article search gives us a wide range of options to query for data, by range of date, fields like day_of_week, headline, subject and more, and news desk values like blogs, cars, fashion and many more. That way we can narrow it down to what we are looking for.

### Example

Here is a link from the above search: https://www.nytimes.com/2018/12/22/us/politics/shutdown-trump-senate.html

![NYTimes HTML Example](images/nytimes_html.png)

As we can see each paragraph is in his own container.

Unfortunately we only get the lead paragraph from our search which doesn't help much if we are looking for the full article. All we get is metadata but that's it.

---

## News API
https://newsapi.org

Get breaking news headlines, and search for articles from over 30,000 news sources and blogs. Similar to the nytimes api we can query for topics we are interested in or just get from all source the top headlines from the past 30 days.

We will get the unformatted content of the article, where available. The content is truncated to 260 characters for the developer plan users, but if you are willing to pay we could get the full content.

In [4]:
# Initialize
newsapi = NewsApiClient(api_key = tokens['newsapi'])
sources = newsapi.get_sources(language='en', country='us')

news_sources = [sources['sources'][i]['id'] for i, source in enumerate(sources['sources'])]
print(news_sources)

['abc-news', 'al-jazeera-english', 'ars-technica', 'associated-press', 'axios', 'bleacher-report', 'bloomberg', 'breitbart-news', 'business-insider', 'buzzfeed', 'cbs-news', 'cnbc', 'cnn', 'crypto-coins-news', 'engadget', 'entertainment-weekly', 'espn', 'espn-cric-info', 'fortune', 'fox-news', 'fox-sports', 'google-news', 'hacker-news', 'ign', 'mashable', 'medical-news-today', 'msnbc', 'mtv-news', 'national-geographic', 'national-review', 'nbc-news', 'new-scientist', 'newsweek', 'new-york-magazine', 'next-big-future', 'nfl-news', 'nhl-news', 'politico', 'polygon', 'recode', 'reddit-r-all', 'reuters', 'techcrunch', 'techradar', 'the-american-conservative', 'the-hill', 'the-huffington-post', 'the-new-york-times', 'the-next-web', 'the-verge', 'the-wall-street-journal', 'the-washington-post', 'the-washington-times', 'time', 'usa-today', 'vice-news', 'wired']


In [5]:
print('Number of source: {}'.format(len(news_sources)))

Number of source: 57


Let us toy around a little and see what results we will get search for "Border" and "Wall".

In [6]:
# define start date
start_year = 2019
start_month = 2
start_day = 24

# define date
date = datetime.date(year = start_year, month = start_month, day = start_day)
sources = ', '.join(news_sources)
all_articles = newsapi.get_everything(sources=sources, from_param=str(date), sort_by='relevancy')
print('Total number of results: {}'.format(all_articles['totalResults']))  

Total number of results: 147791


Let us search for everything possible in all english news sources in the US and sort it by relevance.

In [ ]:
json_file_path = 'newsapi_data'
        
with tqdm() as progress:
    for source in news_sources:
        # we should set the range to the maximum possible number based on the total results devided by page_size
        # articles = newsapi.get_everything(sources=source, from_param=str(date), sort_by='relevancy', page_size=100)
        # articles['totalResults'] // 100 + 1
        for page in range(1, 3):
            articles = newsapi.get_everything(sources=source, from_param=str(date), sort_by='relevancy', page_size=100, page=page)

            try:
                indexes = [i for i, v in enumerate(articles['articles']) if articles['articles'][i]['content'] is None]
                for index in sorted(indexes, reverse=True):
                    del articles['articles'][index]

                date_str = date.strftime("%Y%m%d")
                filename = '_'.join([source, date_str])
                json_file_name = get_json_file(filename, page, json_file_path)
                
                with open(json_file_name, 'w+') as f:
                    f.write(ujson.dumps(articles) + '\n')
                    f.close()
                    
            except Exception as e:
                print(e)
                
        progress.update()

In [13]:
with open('newsapi_data/bloomberg_20190224.1.json') as f:
    data = json.load(f)
    pprint(data['articles'][0])

{'author': 'Katherine Chiglinsky, Michelle Kim',
 'content': 'Judging how Warren Buffett’s Berkshire Hathaway Inc. ended 2018 '
            'very much depended on one’s preferred metric.\r\n'
            'The company posted a staggering net loss of $25 billion in the '
            'fourth quarter, the biggest loss in its history. That was largely '
            'due to a new accoun… [+4462 chars]',
 'description': 'Judging how Warren Buffett’s Berkshire Hathaway Inc. ended '
                '2018 very much depended on one’s preferred metric.',
 'publishedAt': '2019-02-24T17:31:52Z',
 'source': {'id': 'bloomberg', 'name': 'Bloomberg'},
 'title': "Buffett's Stock Losses and Key Takeaways From Berkshire Results",
 'url': 'https://www.bloomberg.com/news/articles/2019-02-24/buffett-s-stock-losses-and-key-takeaways-from-berkshire-results',
 'urlToImage': 'https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iBAm0Fw91hMc/v0/1200x800.jpg'}


Furthermore we could define our query, as well as the domains we are interested in. For example:

In [15]:
url = ('https://newsapi.org/v2/everything?'
       'q=Border+Wall&'
       'domains=wsj.com,nytimes.com&'
       'from=2019-03-24&'
       'sortBy=popularity&'
       'apiKey=%s' % tokens['newsapi'])

response = requests.get(url)
print(response.json()['articles'][0]['content'])

{'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 1,000 requests over a 24 hour period (250 requests available every 6 hours). Please upgrade to a paid plan if you need more requests.'}


News API, if we would pay for it, would be a great tool to gather all sorts of content. Similar to the way the nytimes api works but faster. And we can get content from the nytimes via it `domains` attribute as well.

---

## webhose.io

Another possiblity I found is [webhose](webhose.io). Let us explore how they work and what kind of results we will get back from them.

### API Plyground

Here we have two options to do the scraping. The first and most officious work flow, webhose offers a playground area to fine tune you search and copy and paste the output given including python, java, objective-c and ruby. 

In the example below we search news in the US for the past 30 days, which returns over 2 million articles.

In [3]:
import webhoseio

webhoseio.config(token=tokens['webhoseio'])
query_params = {
    "q": "language:english is_first:true site_type:news thread.country:US site_category:media",
    "ts": "1549870120476",
    "sort": "crawled"
}
output = webhoseio.query("filterWebContent", query_params)
with open('webhose_query.json', 'w') as outfile:
    json.dump(output, outfile)

# Print the text of the first post
print(output['posts'][0]['text'] + '\n')  
# Print the text of the first post publication date
print('Published at {} \n'.format(output['posts'][0]['published']))
# Print the amount of results found
print('Number of results found: {}'.format(output['totalResults']))

Childish Gambino makes Grammy history CNN News February 11, 2019
Childish Gambino’s “This Is America” won Grammys for song and record of the year on Sunday, becoming the first rap song to win the prestigious awards.
The artist — also known as the mutitalented Donald Glover — did not attend the event and reportedly declined an invitation to perform at the Grammys.
“This Is America” beat out “Shallow,”“God’s Plan” and other big hits. The Grammy for song of the year honors song writers, while record of the year goes to the recording artist.
The song caused a stir last May when Gambino released its ambitious video, which was full of racial symbolism.
Related: ‘This Is America’: The Childish Gambino video explained
The Recording Academy has made an effort to diversify its membership amid complaints the Grammys have frequently failed to recognize rap and hip-hop artists in the major categories.
Earlier this week in an interview with the New York Times , longtime Grammy producer Ken Ehrlich a

Our query found over 2 million results for the past 30 days (given as ts), which is a lot. We can narrow down our result by changing the query string. But then we are still have to know of what category the url is. So lets focus on the list provided by opensource.co.

In [12]:
df = pd.read_csv('url_sources.csv')

### Python API

Below we are using the api directly in python and collect the content from the out list of url's. 

In [13]:
# Setting the relative location of the text files
resources_dir = 'url_data'

def collect(filename, query):
    lines = set()

    try:
        # Collect the data from webhose.io with the given query
        response = webhose.search(query)
    except Exception:
        print('Failed to access: {}'.format(query.split(':')[-1]))
        return
    
    next_responce = 1
    while len(response.posts) > 0 and next_responce < 11:
        # Go over the list of posts returned from the response
        for post in response.posts:
            # Verify that the length of the text is not too short
            if len(post.text) > 50:
                # Extracting the text from the post object and clean it
                text = post.text.replace('\n', ' ').replace('\t', ' ')
                
                # add the post-text to the lines we are going to save in the text file
                lines.add(text)
                
        time.sleep(6)
        print('Getting %s' % response.next)
        # Request the next 100 results from webhose.io
        response = response.get_next()
        next_responce += 1

    # Create the text file
    with open(os.path.join(resources_dir, filename + '.txt'), 'a+') as f:
        for line in lines:
            f.write('%s\n' % line)


In [14]:
for i, url in enumerate(df.url):
    query_string = "language:english site:"
    query_url = query_string + url
    
    collect(df.category[i], query_url)

Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format=html&ts=1552260620007&q=language%3Aenglish+site%3Aamericanthinker.com
Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format=html&ts=1552376514024&q=language%3Aenglish+site%3Aamericanthinker.com
Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format=html&ts=1552376521020&q=language%3Aenglish+site%3Aamericanthinker.com
Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format=html&ts=1552376532002&q=language%3Aenglish+site%3Aamericanthinker.com
Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format=html&ts=1552376543005&q=language%3Aenglish+site%3Aamericanthinker.com
Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format=html&ts=1552376562002&q=language%3Aenglish+site%3Aamericanthinker.com
Getting http://webhose.io/search?token=309cd3ce-a298-4970-83f3-8a698919133b&format

Because we only saved the content based on their category without the base url - which is an oversight on our side - we don't know from where each article is from nor how much.

---

## Extracting extra data from a dataset found at webhose.io 

Webhoe offers also archived data which we downloaded. Let's ahve a look and see what we can use from it. To note here is that the content is a few years old. Let us see what we have so far before we will commit to it.

In [24]:
with open('data/news_0499610.json') as f:
    data = json.load(f)
    
data.keys()

dict_keys(['organizations', 'uuid', 'thread', 'author', 'url', 'ord_in_thread', 'title', 'locations', 'entities', 'highlightText', 'language', 'persons', 'text', 'external_links', 'published', 'crawled', 'highlightTitle'])

In [25]:
n_files = len([name for name in os.listdir('data/')])

template = 'data/news_{:07d}.json'
filenames = [template.format(k) for k in range(1, n_files)]

In [26]:
dataframes = (json.load(open(fname)) for fname in filenames if Path(fname).exists())

### URL's

Let's see how many unique url's we have in the data.

In [27]:
url_list = []

with tqdm() as progress:
    for data in dataframes:
        list = tldextract.extract(data['url'])
        domain_name = list.domain + '.' + list.suffix
        url_list.append(domain_name)
        
        progress.update()

499610it [03:21, 2480.59it/s]


In [28]:
urls = set(url_list)
len(urls)

272

In [29]:
url_df = pd.read_csv('url_sources.csv')
url_sources = url_df['url'].values

not_in_source = [url for url in urls if url not in url_sources]
print(not_in_source)

['spotify.com', 'nairaland.com', 'delta.com', 'constantcontact.com', 'bankofamerica.com', 'udn.com', 'slickdeals.net', 'prezi.com', 'gizmodo.com', 'pcmag.com', 'php.net', 'cocolog-nifty.com', 'abplive.in', 'biobiochile.cl', 'office.com', 'marriott.com', 'www.gov.uk', 'moneycontrol.com', 'thekitchn.com', 'mit.edu', 'lemonde.fr', 'cbs.com', 'bbc.com', 'macys.com', 'investopedia.com', 'vimeo.com', 'allrecipes.com', 'xbox.com', 'woot.com', 'quora.com', 'squarespace.com', 'yelp.com', 'conservativetribune.com', 'aol.com', 'telegraph.co.uk', 'bhaskar.com', 'techcrunch.com', 'sourceforge.net', 'booking.com', 'usaa.com', 'babycenter.com', 'airbnb.com', 'patch.com', 'sky.com', 'archive.org', 'att.com', 'howtogeek.com', 'time.com', 'wiley.com', 'rollingstone.com', 'intuit.com', 'wattpad.com', 'speedtest.net', 'adobe.com', 'samsung.com', 'surveymonkey.com', 'ups.com', 'gamespot.com', 'ed.gov', 'icc-cricket.com', '163.com', 'thesportbible.com', 'myfitnesspal.com', 'about.com', 'force.com', 'slate.c

### Select only sites with `.com` suffix

In [30]:
dotcom_list = [url for url in not_in_source if url.endswith('.com')]
len(dotcom_list)

185

### Remove none-US sites

In [31]:
US_list = []
for url in dotcom_list:
    try:
        ip = socket.gethostbyname(url)
        reader = geoip2.database.Reader('GeoLite2-Country_20190305/GeoLite2-Country.mmdb')
        response = reader.country(ip)
        
        if response.country.iso_code == 'US':
            US_list.append(url)
            
    except IOError:
        print('{} nodename nor servname provided, or not known'.format(url))
        
len(US_list)

cocolog-nifty.com nodename nor servname provided, or not known


125

In [33]:
print(US_list)

['spotify.com', 'nairaland.com', 'delta.com', 'constantcontact.com', 'bankofamerica.com', 'prezi.com', 'gizmodo.com', 'pcmag.com', 'thekitchn.com', 'cbs.com', 'investopedia.com', 'allrecipes.com', 'woot.com', 'quora.com', 'squarespace.com', 'conservativetribune.com', 'bhaskar.com', 'techcrunch.com', 'babycenter.com', 'airbnb.com', 'patch.com', 'att.com', 'howtogeek.com', 'time.com', 'wiley.com', 'rollingstone.com', 'wattpad.com', 'adobe.com', 'surveymonkey.com', 'ups.com', 'gamespot.com', 'icc-cricket.com', 'thesportbible.com', 'myfitnesspal.com', 'about.com', 'force.com', 'slate.com', 'windows.com', 'salesforce.com', 'ultimate-guitar.com', 'slack.com', 'worldlifestyle.com', 'houzz.com', 'ted.com', '6pm.com', 'hp.com', 'kooora.com', 'go.com', 'drudgereport.com', 'mcafee.com', 'foodnetwork.com', 'giphy.com', 'ibm.com', 'zillow.com', 'fitbit.com', 'youm7.com', 'alibaba.com', 'apple.com', 'gfycat.com', 'instructure.com', 'ancestry.com', 'vetogate.com', 'evernote.com', 'wired.com', 'devian

Well, looking at the list is an awful lot of site which have nothing to do with news. Let's select the few which are news related.

In [36]:
urls_to_keep = ['westernjournalism.com', 'vox.com', 'time.com', 'thedailybeast.com', 'foxnews.com']

### Collect news article

In [37]:
json_data = (json.load(open(fname)) for fname in filenames if Path(fname).exists())

In [38]:
lines = set()

with tqdm() as progress:
    for data in json_data:
        list = tldextract.extract(data['url'])
        domain_name = list.domain + '.' + list.suffix
        
        if domain_name in urls_to_keep:
            lines.add(data['text'].encode('utf-8'))
        
        progress.update()

499610it [03:22, 2468.63it/s]


In [39]:
len(lines)

4806

We were able to extract 4806 article. 

### Save content

In [40]:
with open(os.path.join(resources_dir, 'political_webhose.txt'), 'a+') as f:
    for line in lines:
        f.write('%s\n' % line)